<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [5]</a>'.</span>

In [1]:
from IPython import get_ipython
from IPython.core.display import display
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
get_ipython().run_line_magic('run', 'setup')

In [2]:
#leeftijdsgroepen = SimpleNamespace()
@run
def cell():
  global rivm
  rivm = RIVM.csv('COVID-19_casus_landelijk')
  display(rivm.head())

downloading rivm/COVID-19_casus_landelijk-2021-12-27@14-15.csv


rivm COVID-19_casus_landelijk zipping rivm/COVID-19_casus_landelijk-2021-12-27@14-15.csv


loading rivm/COVID-19_casus_landelijk-2021-12-27@14-15.csv.gz


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2021-12-27 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2021-12-27 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2021-12-27 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2021-12-27 10:00:00,2020-01-01,DOO,60-69,Female,Noord-Holland,No,No,NaN,GGD Hollands-Noorden
4,2021-12-27 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


In [3]:
@run
def cell():
  global bevolking

  # December 2021:
  aantallenpercohort = {'0-9': 1757736,
    '10-19': 1981885,
    '20-29': 2263186,
    '30-39': 2213705,
    '40-49': 2137833,
    '50-59': 2550358,
    '60-69': 2171183,
    '70-79': 1649291,
    '80-89': 722027,
    '90+': 135289}

  global bevolking

  bevolking = None
  try:
    bevolking = CBS.bevolking(leeftijdsgroepen=True)
  except Exception as e:
    print(e)
    bevolking = None

  if bevolking is None:
    bevolking = aantallenpercohort



In [4]:
# Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.
# vervang <50 en Unknown door Onbekend
@run
def cell():
  rivm['Cohort'] = rivm['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
  # aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
  assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in rivm['Cohort'].unique() if cohort[0].isdigit()]
  assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
  rivm['Gemiddelde leeftijd'] = rivm['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

  # verwijder tijd
  rivm['Date_file_date'] = pd.to_datetime(rivm['Date_file'].replace(r' .*', '', regex=True))

  rivm['Date_statistics_date'] = pd.to_datetime(rivm['Date_statistics'])

  # weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
  rivm['Weken terug'] = np.floor((rivm['Date_file_date'] - rivm['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(int)

  # voeg key, gem leeftijd, kleurnummer en totaal toe
  Date_file = rivm['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
  # check type bevolking
  if type(bevolking) == dict:
    cohorten = list(bevolking.keys())
  else:
    cohorten = list(bevolking.index)
  cohorten += ['Onbekend']

  def summarize(df, category, prefix):
    # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
    # en we het 'rivm' frame later nog clean nodig hebben
    df = df.copy(deep=True)

    df = (df
          .groupby(['Weken terug', 'Cohort'])['count']
          .sum()
          .unstack(fill_value=np.nan)
          .reset_index()
          .rename_axis(None, axis=1)
        ).merge(df
          # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
          # niveau aggregeren voor 'df' overschreven word
          .groupby(['Weken terug'])['Gemiddelde leeftijd']
          .mean()
          .to_frame(), on='Weken terug'
        )

    # altijd 52 rijen
    df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

    # toevoegen missende cohorten
    for col in cohorten:
      if not col in df:
        df[col] = np.nan

    # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen
    df['Totaal'] = df[cohorten].sum(axis=1)

    # voeg periode en datum toe
    # periode afgeleid van weken-terug (= de index voor deze dataframe)
    df['Datum'] = pd.to_datetime(Date_file)
    df['Periode'] = (df
      .index.to_series()
      .apply(
        lambda x: (
          (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
          + '-'
          + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
        )
      )
    )

    # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
    df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
    df['Type'] = category

    # voeg de kleur kolommen toe
    for col in cohorten:
      df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

    # herschikken van de kolommen
    colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
    return df[colorder + [col for col in df if col not in colorder]]

  if type(bevolking) == dict:
    factor = {}
    for c in bevolking: factor[c] = 100_000 / bevolking[c]
  else:
    factor = bevolking.to_dict()['per 100k']

  global tabel
  tabel = pd.concat(
    # flatten the result list zodat pd.concat ze onder elkaar kan plakken
    functools.reduce(lambda a, b: a + b, [
      [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
      for df, label, prefix in [
        (rivm, 'Positief getest', 'p'), # volledige count per cohort
        (rivm[rivm.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
        (rivm[rivm.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
      ]
    ])
  )

  if type(bevolking) == dict:
    totale_bevolking = sum(bevolking.values())
    print(totale_bevolking)
  else:
    totale_bevolking_per_cohort = bevolking.to_dict()['BevolkingOpDeEersteVanDeMaand']
    totale_bevolking = sum( totale_bevolking_per_cohort.values())

  keys = [ key for key in tabel["Key"]]
  totals = [ x for x in tabel["Totaal"]]
  for k in range(0, len(keys)):
    key = keys[k]
    if '100k' in key:
      abskey = key[0] + key[5:]
      kk = keys.index(abskey)
      if isinstance(totals[k], float):
        correctedtotal = totals[kk] * (100_000 / totale_bevolking)
        # print([k, totals[k], totals[kk], correctedtotal])
        totals[k] = correctedtotal

  tabel["Totaal"] = totals


  # rood -> groen
  cdict = {
    'red':   ((0.0, 0.0, 0.0),   # no red at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
    'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0)),  # no green at 1
    'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
              (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
              (1.0, 0.0, 0.0))   # no blue at 1
  }
  cm = colors.LinearSegmentedColormap('GnRd', cdict)
  # geel -> paars
  cm = sns.color_palette('viridis_r', as_cmap=True)
  display(tabel
    .fillna(0)
    .head()
    .round(1)
    .reset_index(drop=True)
    .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
  )

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2021-12-27 00:00:00,21/12-27/12,35.400000,48556.000000,Positief getest,5409.000000,6365.000000,9281.000000,8518.000000,7144.000000,5720.000000,3670.000000,1704.000000,572.000000,173.000000,0.000000,582,685,1000,917,769,616,395,183,61,18,0
1,p001,1,2021-12-27 00:00:00,14/12-20/12,35.900000,88169.000000,Positief getest,11615.000000,12557.000000,12906.000000,14887.000000,12978.000000,10215.000000,7152.000000,3858.000000,1532.000000,468.000000,1.000000,780,843,866,1000,871,686,480,259,102,31,0
2,p002,2,2021-12-27 00:00:00,07/12-13/12,36.400000,108301.000000,Positief getest,15669.000000,16157.000000,12950.000000,17478.000000,16034.000000,11844.000000,9123.000000,5477.000000,2736.000000,830.000000,3.000000,896,924,740,1000,917,677,521,313,156,47,0
3,p003,3,2021-12-27 00:00:00,30/11-06/12,36.600000,136067.000000,Positief getest,19149.000000,20747.000000,16034.000000,21840.000000,20453.000000,15074.000000,11297.000000,6779.000000,3609.000000,1082.000000,3.000000,876,949,734,1000,936,690,517,310,165,49,0
4,p004,4,2021-12-27 00:00:00,23/11-29/11,36.900000,148442.000000,Positief getest,19962.000000,23594.000000,17199.000000,22893.000000,22231.000000,16877.000000,12567.000000,7674.000000,4153.000000,1288.000000,4.000000,846,1000,728,970,942,715,532,325,176,54,0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [5]:
if knack:
  await knack.publish(tabel.fillna(0).assign(Datum=tabel.Datum.dt.strftime('%Y-%m-%d')), 'Leeftijdsgroep', Cache)

infinities:


Empty DataFrame
Columns: []
Index: []


nan:


Empty DataFrame
Columns: []
Index: []


Key                     object
Weken terug              int64
Datum                   object
Periode                 object
Gemiddelde leeftijd    float64
Totaal                 float64
Type                    object
0-9                    float64
10-19                  float64
20-29                  float64
30-39                  float64
40-49                  float64
50-59                  float64
60-69                  float64
70-79                  float64
80-89                  float64
90+                    float64
Onbekend               float64
c0-9                     int64
c10-19                   int64
c20-29                   int64
c30-39                   int64
c40-49                   int64
c50-59                   int64
c60-69                   int64
c70-79                   int64
c80-89                   int64
c90+                     int64
cOnbekend                int64
dtype: object
updating knack


...

 95%|█████████▍| 140/148 [00:22<00:01,  5.55it/s]

 95%|█████████▌| 141/148 [00:22<00:01,  5.03it/s]

 96%|█████████▌| 142/148 [00:23<00:01,  5.21it/s]

 99%|█████████▊| 146/148 [00:23<00:00,  4.92it/s]

TimeoutError: 